In [1]:
import os

import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
# Set current working directory
os.chdir("/import/home2/yhchenmath/Code/project5472/")
root_dir = "/import/home2/yhchenmath/Dataset/stitch3D/HumanHeart/"

## ST

In [12]:
count = pd.read_csv(f"{root_dir}SC/Filtered/filtered_ST_matrix_and_meta_data/filtered_matrix.tsv", sep="\t", index_col=0).T
meta = pd.read_csv(f"{root_dir}SC/Filtered/filtered_ST_matrix_and_meta_data/meta_data.tsv", sep="\t", index_col=0)
genes_new = pd.read_csv(f"{root_dir}genes_new.txt")

count.columns = list(genes_new.x.values)
count = count.loc[:, count.columns.notna()]

In [13]:
count

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AC009802.1,AC112487.2,AC068587.4,RF00537,MYOCOS,AC013268.5,AC018553.2,Z96074.1,AL592295.4,AC012442.3
1x17x20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1x17x21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1x17x22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1x18x18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1x18x19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19x9x11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19x9x12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19x9x13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19x9x8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from matplotlib.image import imread
adata_st_list_raw = []

for i in range(1, 10):
    count_i = count[[loc.split("x")[0]==str(i+4) for loc in count.index]]
    count_i.index = [(loc.split("x")[1]+"x"+loc.split("x")[2]) for loc in count_i.index]
    meta_i = meta[[loc.split("x")[0]==str(i+4) for loc in meta.index]]
    meta_i.index = [(loc.split("x")[1]+"x"+loc.split("x")[2]) for loc in meta_i.index]

    loc_i = pd.read_csv(root_dir + "ST/ST_Samples_6.5PCW/ST_Sample_6.5PCW_%d/spot_data-all-ST_Sample_6.5PCW_%d.tsv" % (i, i),
                       sep="\t")

    loc_i.index = [(str(loc_i.x.values[k]) + 'x' + str(loc_i.y.values[k])) for k in range(loc_i.shape[0])]
    loc_i = loc_i.loc[meta_i.index]

    img_i = imread(root_dir + 'ST/ST_Samples_6.5PCW/ST_Sample_6.5PCW_%d/ST_Sample_6.5PCW_%d_HE_small.jpg' % (i, i))

    adata_st_i = ad.AnnData(X = count_i.values)

    adata_st_i.obs = meta_i
    adata_st_i.obs['selected'] = loc_i['selected'].values
    adata_st_i.var.index = count_i.columns

    library_id = '0'
    adata_st_i.uns["spatial"] = dict()
    adata_st_i.uns["spatial"]['0'] = dict()
    adata_st_i.uns["spatial"]['0']['images'] = dict()
    adata_st_i.uns["spatial"]['0']['images']['hires'] = img_i
    adata_st_i.uns["spatial"]['0']['scalefactors'] = {'spot_diameter_fullres': 100,
                                                      'tissue_hires_scalef': 1.0,
                                                      'fiducial_diameter_fullres': 100,
                                                      'tissue_lowres_scalef': 1.0}

    adata_st_i.obsm['spatial'] = np.concatenate((loc_i['pixel_x'].values.reshape(-1, 1),
                                                 loc_i['pixel_y'].values.reshape(-1, 1)), axis=1)

    adata_st_i.obsm['loc_use'] = np.concatenate((loc_i['x'].values.reshape(-1, 1),
                                                 loc_i['y'].values.reshape(-1, 1)), axis=1)

    adata_st_i.obs['array_row'] = loc_i['y'].values
    adata_st_i.obs['array_col'] = loc_i['x'].values

    adata_st_i = adata_st_i[adata_st_i.obs['selected'].values != 0]
    adata_st_list_raw.append(adata_st_i.copy())

/tmp/ipykernel_942185/615017188.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata_st_i.obs['selected'] = loc_i['selected'].values
/tmp/ipykernel_942185/615017188.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata_st_i.obs['array_row'] = loc_i['y'].values
/tmp/ipykernel_942185/615017188.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

## SC

In [15]:
count_ref = pd.read_csv(f"{root_dir}SC/Filtered/share_files/all_cells_count_matrix_filtered.tsv",
                        sep='\t', index_col=0).T

meta_ref = pd.read_csv(f"{root_dir}SC/Filtered/share_files/all_cells_meta_data_filtered.tsv",
                        sep='\t', index_col=0)

adata_ref = ad.AnnData(X = count_ref.values)
adata_ref.obs.index = count_ref.index
adata_ref.var.index = count_ref.columns
for col in meta_ref.columns[:-1]:
    adata_ref.obs[col] = meta_ref.loc[count_ref.index][col].values

In [16]:
adata_ref

AnnData object with n_obs × n_vars = 3777 × 15323
    obs: 'nGene', 'nUMI', 'experiment', 'Phase', 'res.0.7', 'celltype'

## Save in pickle

In [23]:
adata_st_list_raw[0]

AnnData object with n_obs × n_vars = 100 × 39525
    obs: 'nGene', 'nUMI', 'Sample', 'weeks', 'ChipBatch', 'ChipNr', 'Experiment_date', 'Experiment_procedure', 'Sequencing_date', 'Raw_reads', 'new_x', 'new_y', 'percent.mito', 'res.0.8', 'selected', 'array_row', 'array_col'
    uns: 'spatial'
    obsm: 'spatial', 'loc_use'

In [22]:
# Save adata_st_list_raw and adata_ref in pickle
# para_dict: the extra parameters used in the pipeline
import pickle
if not os.path.exists(f"{root_dir}process"):
    os.makedirs(f"{root_dir}process")
with open(f"{root_dir}process/st_data.pickle", 'wb') as f:
    pickle.dump(adata_st_list_raw, f)
with open(f"{root_dir}process/sc_data.pickle", 'wb') as f:
    pickle.dump(adata_ref, f)
with open(f"{root_dir}process/para_dict.pickle", 'wb') as f:
    pickle.dump({"preprocess": {"slice_dist_micron": [5., 115., 85., 160.,5., 160., 5., 155.,]},
                 "align": {}
                 }, f)